In [9]:
from pathlib import Path
import requests
import pandas as pd

data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [107]:
for (columnName, columnData) in data.iteritems():
    def normalize(y, x):
        maximum = x.max()
        minimum = x.min()
        x = (x-minimum)/(maximum - minimum)
        data[y] = x
    normalize(columnName, columnData)
        
data.head()
data.shape


(303, 14)

In [179]:
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.optim import *
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os
import numpy as np
import random


target = pd.DataFrame(data = data)
torch_tensor = torch.tensor(target.values)
random.shuffle(torch_tensor)
train_size = int(0.8 * len(torch_tensor))
test_size = len(torch_tensor) - train_size

train_set = torch_tensor[:train_size, :]
test_set = torch_tensor[:test_size, :]

train_health = train_set[:, :-1]
train_labels = train_set[:, -1]

test_health = test_set[:, :-1]
test_labels = test_set[:, -1]


train_data = []
for i in range(len(train_set)):
    train_data.append([train_health[i], train_labels[i]])

test_data = []
for i in range(len(test_set)):
    train_data.append([test_health[i], test_labels[i]])

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
               
               
dataiter = iter(trainloader)
health, labels = dataiter.next()

print(health.shape)
print(labels.shape)

torch.Size([64, 13])
torch.Size([64])


In [159]:
input_size = 13
hidden_sizes = [10, 10]
output_size = 2

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]), 
                      nn.ReLU(), 
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]), 
                      nn.ReLU(), 
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))

print(model)
print(train_labels.shape)

Sequential(
  (0): Linear(in_features=13, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=10, bias=True)
  (3): ReLU()
  (4): Linear(in_features=10, out_features=2, bias=True)
  (5): LogSoftmax(dim=1)
)
torch.Size([242])


In [164]:
criterion = nn.NLLLoss()
health, labels = next(iter(trainloader))
labels = labels.long()
log_probability = model(health.float())
loss = criterion(log_probability, labels)

# print(model[0].weight.grad)
loss.backward()
# print(model[0].weight.grad)

In [174]:
import torch.optim as optim
from time import time

optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
epochs = 20
total_time = time()
for e in range(epochs):
    running_loss = 0
    for health, labels in trainloader:        
        optimizer.zero_grad()
        output = model(health.float())
        loss = criterion(output, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}.".format(e, running_loss/len(trainloader)))
        
print("\nTraining Time(in minutes) =", (time()-total_time)/60)

Epoch 0 - Training loss: 0.16896067559719086.
Epoch 1 - Training loss: 0.17556948214769363.
Epoch 2 - Training loss: 0.16559593193233013.
Epoch 3 - Training loss: 0.16559774801135063.
Epoch 4 - Training loss: 0.17560715600848198.
Epoch 5 - Training loss: 0.1653027143329382.
Epoch 6 - Training loss: 0.16852409206330776.
Epoch 7 - Training loss: 0.16534443572163582.
Epoch 8 - Training loss: 0.16867654398083687.
Epoch 9 - Training loss: 0.1686637457460165.
Epoch 10 - Training loss: 0.1683270400390029.
Epoch 11 - Training loss: 0.16849876567721367.
Epoch 12 - Training loss: 0.17187050729990005.
Epoch 13 - Training loss: 0.16501137055456638.
Epoch 14 - Training loss: 0.1717620361596346.
Epoch 15 - Training loss: 0.16159043461084366.
Epoch 16 - Training loss: 0.16802867129445076.
Epoch 17 - Training loss: 0.17168916389346123.
Epoch 18 - Training loss: 0.17138873599469662.
Epoch 19 - Training loss: 0.16811328195035458.

Training Time(in minutes) = 0.001478882630666097


In [185]:
correct_count, all_count = 0, 0
output_probabilities = []

for health, labels in testloader:
    for i in range(len(labels)):
        with torch.no_grad():
            log_probabilities = model(health.float())
        ps = torch.exp(log_probabilities)
        output_probabilities.append(ps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.numpy()[i]
        if(true_label == pred_label):
            correct_count += 1
        all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))
print(ps.numpy()[0])


Number Of Images Tested = 303

Model Accuracy = 0.9438943894389439
[0.0412243  0.95877576]
